In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import utils

In [2]:
t0_csv = "/workspaces/seed/paper/sanhMultitaskPromptedTraining2022a/evaluation_result/20230114/T0/T0.csv"
t0_failed = "/workspaces/seed/paper/sanhMultitaskPromptedTraining2022a/evaluation_result/20230114/T0/hellaswag/None/complete_first_then.json"

t0 = pd.read_csv(t0_csv)
t0["subset_name"] = t0["subset_name"].fillna("all")

utils.report(t0)


accuracy                                          
                                 max       min      mean       std num_prompts
dataset_name subset_name                                                      
super_glue   wsc.fixed      0.692308  0.009615  0.523077  0.207082          10
             copa           0.960000  0.320000  0.810459  0.189014          12
             wic            0.620000  0.035000  0.485500  0.164104          10
             cb             0.803571  0.392857  0.722619  0.106261          15
anli         all            0.445000  0.225000  0.392333  0.057317          15
super_glue   rte            0.875000  0.750000  0.827000  0.043856          10
winogrande   winogrande_xl  0.630000  0.550000  0.595000  0.032787           5
hellaswag    all            0.040000  0.020000  0.030000  0.009129           4

### `hellaswag` is multiple choice, sentence completion task
- The author use sequence log prob to deal with multiple choice eval. 
- [HF example code](https://huggingface.co/docs/transformers/tasks/multiple_choice) is ugly and messy. 
- It would be helpful to learn how to calculate seq prob, but that won't work with benchmarking openai api. 


In [3]:
utils.read_failed_cases(t0_failed)


input: Complete the description with an appropriate ending:
First, [header] how to recover from an emotional affair [title] forgive yourself. [step] while forgiving others can be challenging, it's often even harder to forgive yourself. remember that if you had known the path of your actions and their consequences, you probably would not have done what you did. Then,  ...

(a) Take some time to live in the past and let go of those emotions. For example, if you had experienced a miscarriage, forgiveness would be easy.

(b) Forgiveness in someone else can only serve to make it harder. [substeps] Cheating can be very emotional, and can even be worse.

(c) To begin forgiving yourself, admit that you messed up or made a mistake. Making mistakes is part of being human and no one is exempt from it.

(d) In the moment, forgive yourself for all the things you could have done differently. [substeps] Though you may have felt wronged, forgiving yourself for your actions will take time and effort to

In [4]:
hellaswag = utils.load_raw_dataset("hellaswag", None)


[2023-01-16 02:06:40,705] [datasets.builder] [builder.py:785] Found cached dataset hellaswag (/workspaces/seed/cache/hf_dataset/hellaswag/default/0.1.0/c37cd37196278995f42bc32f532730ae9b0d5f0f4a2d3b97735c17ff3ad67169)


In [5]:
hellaswag[0]


{'ind': 24,
 'activity_label': 'Roof shingle removal',
 'ctx_a': 'A man is sitting on a roof.',
 'ctx_b': 'he',
 'ctx': 'A man is sitting on a roof. he',
 'endings': ['is using wrap to wrap a pair of skis.',
  'is ripping level tiles off.',
  "is holding a rubik's cube.",
  'starts pulling up roofing on a roof.'],
 'source_id': 'activitynet~v_-JhWjGDPHMY',
 'split': 'val',
 'split_type': 'indomain',
 'label': '3'}

In [6]:
utils.peak(t0, "hellaswag")


,checkpoint,dataset_name,subset_name,test_size,time,prompt_name,accuracy
65,bigscience/T0,hellaswag,all,200,21.490077,Predict ending with hint,0.020
66,bigscience/T0,hellaswag,all,200,23.227984,Randomized prompts template,0.040
67,bigscience/T0,hellaswag,all,200,26.130677,complete_first_then,0.025
68,bigscience/T0,hellaswag,all,200,61.362784,if_begins_how_continues,0.035


In [7]:
prompt = utils.get_prompt("hellaswag", None, "complete_first_then")


In [8]:
q, a = prompt.apply(hellaswag[0])
print(q)


Complete the description with an appropriate ending:
First, a man is sitting on a roof. Then, he ...

(a) is using wrap to wrap a pair of skis.

(b) is ripping level tiles off.

(c) is holding a rubik's cube.

(d) starts pulling up roofing on a roof.


In [9]:
t2t = utils.build_t2t("bigscience/T0")


[2023-01-16 02:06:41,062] [benchmark] [utils.py:25] loading model from bigscience/T0...


In [10]:
input = """\
Complete the description with an appropriate ending:
First, a man is sitting on a roof. Then, he ...

(a) is using wrap to wrap a pair of skis.

(b) is ripping level tiles off.

(c) is holding a rubik's cube.

(d) starts pulling up roofing on a roof.

Which one is the best answer? Respond with a, b, c, or d. 
"""


In [11]:
t2t(input)


['d']

In [37]:
# try manualy bench hellaswag
prompts = [
        "Predict ending with hint",
        "Randomized prompts template",
        "complete_first_then",
        "if_begins_how_continues",
        ]


cutoff = 200

for prompt_template in prompts:
    correct = 0
    prompt = utils.get_prompt('hellaswag', None, prompt_template)
    
    for instance in hellaswag.select(range(cutoff)):
        q, _ = prompt.apply(instance)
        q = q + "\nWhich one is the best answer? Respond with a, b, c, or d."
        a = t2t(q)[0]

        if len(a) > 1:
            a = -1
        else:
            a = ord(a) - ord('a')
        
        target = int(instance['label'])
        
        if a == target:
                correct += 1

    print(f'{prompt.name}: accuracy: {correct / cutoff}')
    

Predict ending with hint: accuracy: 0.43
Randomized prompts template: accuracy: 0.365
complete_first_then: accuracy: 0.39
if_begins_how_continues: accuracy: 0.365


This way of evaluation is not only eval on langauge modeling, but the result of instruction finetuning. Not ideal but the result is close to official number, around 38% acc. 